In [1]:
import folium
import pandas as pd

m = folium.Map(tiles="cartodb positron")
m.save("footprint.html")

In [2]:
f='./data/ar_ga_tx_combined_coo_zip.csv'
from pandas_geojson import to_geojson
geo_df=pd.read_csv(f)
geo_json = to_geojson(df=geo_df, lat='latitude', lon='longitude',
                 properties=['zip','population','city', 'county','state'])
print(geo_json)

{'type': 'FeatureCollection', 'features': [{'type': 'Feature', 'properties': {'zip': 72712, 'population': 60096, 'city': 'Bentonville', 'county': 'Benton', 'state': 'AR'}, 'geometry': {'type': 'Point', 'coordinates': [-94.25628, 36.346567]}}, {'type': 'Feature', 'properties': {'zip': 72401, 'population': 59098, 'city': 'Jonesboro', 'county': 'Craighead', 'state': 'AR'}, 'geometry': {'type': 'Point', 'coordinates': [-90.656774, 35.886344]}}, {'type': 'Feature', 'properties': {'zip': 72764, 'population': 57561, 'city': 'Springdale', 'county': 'Washington', 'state': 'AR'}, 'geometry': {'type': 'Point', 'coordinates': [-94.060223, 36.175682]}}, {'type': 'Feature', 'properties': {'zip': 72034, 'population': 50619, 'city': 'Conway', 'county': 'Faulkner', 'state': 'AR'}, 'geometry': {'type': 'Point', 'coordinates': [-92.486049, 35.049921]}}, {'type': 'Feature', 'properties': {'zip': 71913, 'population': 47190, 'city': 'Hot Springs National Park', 'county': 'Garland', 'state': 'AR'}, 'geometry

In [3]:
from pandas_geojson import write_geojson
write_geojson(geo_json, filename='three_states.geojson', indent=4)

In [4]:
f='./data/ar_ga_tx_combined_coo_zip.csv'
zip_pop_df=pd.read_csv('./data/ar_ga_tx_combined_coo_zip.csv')
zip_pop_df_dict=zip_pop_df[['zip', 'population']]
zip_pop_df_dict

,zip,population
0,72712,60096
1,72401,59098
2,72764,57561
3,72034,50619
4,71913,47190
...,...,...
2908,79834,202
2909,78545,202
2910,79003,202
2911,79011,202


In [5]:
import branca.colormap as cm
from branca.colormap import linear
# zip_pop_df=pd.read_csv('ar_ga_tx_combined_coo_zip.csv')
colormap = linear.YlGn_09.scale(
    zip_pop_df_dict.population.min(), zip_pop_df_dict.population.max()
)

print(colormap(5.0))

colormap

#ffffe5ff


In [6]:
import folium
import json

geo_json_data = json.load(open('./data/us-states.json'))
map = folium.Map(location=(31.0902, -99.7129), zoom_start=3, tiles="openstreetmap")
map.choropleth(geo_data=geo_json_data, data=zip_pop_df,
             columns=['state', 'population'],
             key_on='feature.id',
             fill_color='YlGn', fill_opacity=0.7, line_opacity=0.2,
             legend_name='Participation Rate (%)')
map

C:\Users\Anyuszi\AppData\Roaming\Python\Python39\site-packages\folium\folium.py:465: FutureWarning: The choropleth  method has been deprecated. Instead use the new Choropleth class, which has the same arguments. See the example notebook 'GeoJSON_and_choropleth' for how to do this.
  warnings.warn(


In [8]:

url = ('./data/three_states.geojson')
max_pop=zip_pop_df['population'].max()
m=folium.Map(location=(31.0902, -99.7129), zoom_start=3, tiles="openstreetmap")
folium.Choropleth(
    geo_data=url,
    data=zip_pop_df_dict,
    columns=("zip", "population"),
    key_on="feature.properties.city",
    style_function=lambda feature: {
        "fillColor": colormap(zip_pop_df_dict[feature["id"]]),
        "color": "black",
        "weight": 1,
        "dashArray": "5, 5",
        "fillOpacity": 0.9,
    },

    legend_name="Population By Zipcode",
    name="States"
).add_to(m)

folium.LayerControl().add_to(m)

tooltip = "Click me!"

folium.Marker(
    [31.0902, -99.7129], 
    tooltip=tooltip
).add_to(m)

for row in zip_pop_df.to_records():
    pos = (row['latitude'],row['longitude'])
    marker = folium.Marker(location=pos, 
                    popup="zip:%s, city:%s, county:%s, state:%s, population:%s" % (row['zip'],row['city'], row['county'],row['state'], row['population'])
                          )
    m.add_child(marker)



m.save("three_states.html")

